# logger

In [7]:
import logging

def my_function(logger) -> float:
    try:
        return 10 / 0
    except Exception as e:
        logger.info(f'an error occured: {str(e)}', exc_info=True)

# dummy main
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
my_function(logger)

INFO:__main__:an error occured: division by zero
Traceback (most recent call last):
  File "/tmp/ipykernel_178713/950500891.py", line 5, in my_function
    return 10 / 0
           ~~~^~~
ZeroDivisionError: division by zero


In [12]:
import inspect

class MyClass:
    def my_method(self):
        """
        This is the docstring of my_method.
        """
        return "Hello, World!"

    def get_method_docstring(self):
        method_name = inspect.currentframe().f_code.co_name
        print(method_name)
        docstring = inspect.getdoc(getattr(self, method_name))
        print(docstring)
        return docstring[:-1] if docstring else None

# dummy main
obj = MyClass()
docstring = obj.get_method_docstring()
print(docstring)

get_method_docstring
None
None


In [13]:
class MyClass:
    def my_method(self):
        method_name = inspect.currentframe().f_code.co_name
        print(method_name)

obj = MyClass()
obj.my_method()

my_method


In [17]:
import logging
import inspect

def log_error(logger, e):
    calling_function = inspect.currentframe().f_back.f_code.co_name
    #logger.error("An error occurred in function %s: %s", calling_function, str(e), exc_info=True)
    logger.error(f"An error occurred in function {calling_function}: {str(e)}", exc_info=True)

def my_function(logger):
    try:
        # 何らかの処理
        result = 10 / 0  # 例外を発生させる例
    except Exception as e:
        # エラー情報をログに出力
        log_error(logger, e)


# dummy main
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
my_function(logger)

ERROR:__main__:An error occurred in function my_function: division by zero
Traceback (most recent call last):
  File "/tmp/ipykernel_178713/3047794893.py", line 12, in my_function
    result = 10 / 0  # 例外を発生させる例
             ~~~^~~
ZeroDivisionError: division by zero


## logger引き渡し検証

In [19]:
import logging

def my_function(logger, other_param1, other_param2):
    logger.info('Logging some information')

# dummy main
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

my_function(logger, 10, 20)


INFO:__main__:Logging some information


In [23]:
# decoratorバージョン
import logging
from functools import wraps

def log_function(logger):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            logger.info(f'calling function {func.__name__}')
            result = func(logger, *args, **kwargs)
            logger.info(f'{func.__name__} completed')
            return result
        return wrapper
    return decorator

@log_function(logger=logging.getLogger(__name__))
def my_function(logger, other_param1, other_param2):
    logger.info(f'{other_param1} {other_param2}')

# dummy main
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

my_function(1, 2)

INFO:__main__:calling function my_function
INFO:__main__:1 2
INFO:__main__:my_function completed


In [ ]:
# log_utils.py
import logging
from functools import wraps

def log_method(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        logger = getattr(args[0], "logger", logging.getLogger(__name__))
        try:
            logger.info(f"Calling {func.__name__}")
            result = func(*args, **kwargs)
            logger.info(f"{func.__name__} completed")
            return result
        except Exception as e:
            logger.error(f"An error occurred in {func.__name__}: {str(e)}", exc_info=True)
    return wrapper

# my_module.py
from log_utils import log_method

class MyClass:
    def __init__(self, logger=None):
        self.logger = logger or logging.getLogger(__name__)

    @staticmethod
    @log_method
    def my_static_method(other_param1, other_param2):
        # 何らかの処理
        result = other_param1 / other_param2
        return result

    @classmethod
    @log_method
    def my_class_method(cls, other_param1, other_param2):
        # 何らかの処理
        result = other_param1 * other_param2
        return result

    @log_method
    def my_instance_method(self, other_param1, other_param2):
        # 何らかの処理
        result = other_param1 + other_param2
        return result


# main_module.py
import logging
from my_module import MyClass

def main():
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger(__name__)

    # インスタンスを作成
    my_instance = MyClass(logger=logger)

    # 各メソッドを呼び出す
    MyClass.my_static_method(10, 2)
    MyClass.my_class_method(5, 3)
    my_instance.my_instance_method(3, 7)

if __name__ == "__main__":
    main()


